# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since you have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [23]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [24]:
df = pd.read_csv("Dataframes/economic_story.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,Retailing,3.843706,1.259813e+10,1.615339e+09,1.406160e+10,...,2.27273,2.200,189965000.0,360.11111,24.95082,318.21,2.383926,-9.164375,238539.0,4.196
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,1.556187e+10,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,2.749481,-21.661570,185584.0,1.503
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,1.843682e+10,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,3.422880,-25.160664,290897.0,34.342
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,2.384882e+10,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,4.504382,30.489705,82349.0,40.267
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,1.769954e+10,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,3.684400,-5.130322,157740.0,17.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0,1.471
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0,8.095
20117,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0,1.068
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,5.459128e+09,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.895787,24.148726,0.0,2.482


In [25]:
#df.dtypes

In [26]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [27]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,Retailing,3.843706,1.259813e+10,1.615339e+09,1.406160e+10,...,2.27273,2.200,189965000.0,360.11111,24.95082,318.21,2.383926,-9.164375,238539.0,4.196
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,1.405795e+10,2.055818e+09,1.556187e+10,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,2.749481,-21.661570,185584.0,1.503
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,1.696712e+10,1.412650e+09,1.843682e+10,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,3.422880,-25.160664,290897.0,34.342
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,2.268979e+10,1.035557e+09,2.384882e+10,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,4.504382,30.489705,82349.0,40.267
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,1.742020e+10,1.411448e+09,1.769954e+10,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,3.684400,-5.130322,157740.0,17.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,4.877168e+09,1.583900e+09,5.553068e+09,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.920659,16.003852,157761.0,1.471
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,4.273414e+09,1.504900e+09,5.029514e+09,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.903176,2.467620,236291.0,8.095
20117,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,4.258229e+09,1.552300e+09,5.436229e+09,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.906078,-0.054218,0.0,1.068
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,4.309428e+09,1.498900e+09,5.459128e+09,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.895787,24.148726,0.0,2.482


Standardising

In [28]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,Retailing,3.843706,0.004337,0.065637,0.031045,...,2.27273,2.200,189965000.0,360.11111,24.95082,318.21,0.029064,0.235367,0.001361,4.196
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,0.004842,0.068358,0.031540,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,0.029495,0.195817,0.001059,1.503
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,0.005848,0.064385,0.032488,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,0.030290,0.184744,0.001660,34.342
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,0.007826,0.062055,0.034274,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,0.031566,0.360861,0.000470,40.267
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,0.006004,0.064377,0.032245,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,0.030598,0.248134,0.000900,17.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,0.001667,0.065443,0.028238,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.027338,0.315017,0.000900,1.471
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,0.001459,0.064955,0.028065,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.027317,0.272179,0.001348,8.095
20117,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,0.001453,0.065247,0.028199,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.027321,0.264198,0.000000,1.068
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,0.001471,0.064917,0.028207,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.027308,0.340793,0.000000,2.482


In [29]:
df_panel = df_clean.copy()
df_panel

,Instrument,Date,Earnings Per Share - Actual,Earnings Per Share - Mean,Earnings Per Share - Actual Surprise,GICS Industry Group Name,Earnings Per Share – Coefficient of Variation,Market Capitalization,Revenue - Actual,Enterprise Value,...,Recommendation - Mean (1-5),Recommendation - Mean (1-5).1,Net Income after Tax,Price Target - Mean,Price Target - Standard Deviation,Price Close,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,Earnings Per Share - Actual Surprise AbsVals
0,POOL.OQ,2022-10-01,4.78,4.58750,4.196,Retailing,3.843706,0.004337,0.065637,0.031045,...,2.27273,2.200,189965000.0,360.11111,24.95082,318.21,0.029064,0.235367,0.001361,4.196
1,POOL.OQ,2022-07-01,7.63,7.51700,1.503,Retailing,3.625249,0.004842,0.068358,0.031540,...,2.20000,2.000,307205000.0,435.37500,57.57373,351.23,0.029495,0.195817,0.001059,1.503
2,POOL.OQ,2022-04-01,4.23,3.14867,34.342,Retailing,6.630736,0.005848,0.064385,0.032488,...,2.00000,2.300,179203000.0,519.88889,37.49206,422.85,0.030290,0.184744,0.001660,34.342
3,POOL.OQ,2022-01-01,2.63,1.87500,40.267,Retailing,3.269867,0.007826,0.062055,0.034274,...,2.30000,2.300,107542000.0,571.00000,41.33833,566.00,0.031566,0.360861,0.000470,40.267
4,POOL.OQ,2021-10-01,4.51,3.84833,17.194,Retailing,2.836295,0.006004,0.064377,0.032245,...,2.30000,2.300,184573000.0,545.28571,50.48823,434.41,0.030598,0.248134,0.000900,17.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20115,AVY.N,2014-01-01,0.69,0.68000,1.471,Materials,3.204412,0.001667,0.065443,0.028238,...,2.33333,2.625,42100000.0,54.62500,3.42555,50.19,0.027338,0.315017,0.000900,1.471
20116,AVY.N,2013-10-01,0.69,0.63833,8.095,Materials,2.775994,0.001459,0.064955,0.028065,...,2.62500,2.875,62000000.0,47.71429,3.45230,43.52,0.027317,0.272179,0.001348,8.095
20117,AVY.N,2013-07-01,0.71,0.70250,1.068,Materials,1.551601,0.001453,0.065247,0.028199,...,2.87500,2.875,70800000.0,45.00000,4.30946,42.76,0.027321,0.264198,0.000000,1.068
20118,AVY.N,2013-04-01,0.59,0.57571,2.482,Materials,2.046169,0.001471,0.064917,0.028207,...,2.87500,3.000,66800000.0,42.57143,3.92272,43.07,0.027308,0.340793,0.000000,2.482


#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [30]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

Earnings Per Share - Actual  Earnings Per Share - Mean  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                         4.78                    4.58750   
           2022-07-01                         7.63                    7.51700   
           2022-04-01                         4.23                    3.14867   
           2022-01-01                         2.63                    1.87500   
           2021-10-01                         4.51                    3.84833   
...                                            ...                        ...   
AVY.N      2014-01-01                         0.69                    0.68000   
           2013-10-01                         0.69                    0.63833   
           2013-07-01                         0.71                    0.70250   
           2013-04-01                         0.59                    0.57571   
           2013-01-01                         0.54                    0.48571   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
POOL.OQ    2022-10-01                                 4.196   
           2022-07-01                                 1.503   
           2022-04-01                                34.342   
           2022-01-01                                40.267   
           2021-10-01                                17.194   
...                                                     ...   
AVY.N      2014-01-01                                 1.471   
           2013-10-01                                 8.095   
           2013-07-01                                 1.068   
           2013-04-01                                 2.482   
           2013-01-01                                11.178   

                      GICS Industry Group Name  \
Instrument Date                                  
POOL.OQ    2022-10-01                Retailing   
           2022-07-01                Retailing   
           2022-04-01                Retailing   
           2022-01-01                Retailing   
           2021-10-01                Retailing   
...                                        ...   
AVY.N      2014-01-01                Materials   
           2013-10-01                Materials   
           2013-07-01                Materials   
           2013-04-01                Materials   
           2013-01-01                Materials   

                       Earnings Per Share – Coefficient of Variation  \
Instrument Date                                                        
POOL.OQ    2022-10-01                                       3.843706   
           2022-07-01                                       3.625249   
           2022-04-01                                       6.630736   
           2022-01-01                                       3.269867   
           2021-10-01                                       2.836295   
...                                                              ...   
AVY.N      2014-01-01                                       3.204412   
           2013-10-01                                       2.775994   
           2013-07-01                                       1.551601   
           2013-04-01                                       2.046169   
           2013-01-01                                       3.625620   

                       Market Capitalization  Revenue - Actual  \
Instrument Date                                                  
POOL.OQ    2022-10-01               0.004337          0.065637   
           2022-07-01               0.004842          0.068358   
           2022-04-01               0.005848          0.064385   
           2022-01-01               0.007826          0.062055   
           2021-10-01               0.006004          0.064377   
...                                      ...               ...   
AVY.N      2014-01-01            

In [31]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


C:\Users\alexa\mambaforge\envs\AnalystForecast\Lib\site-packages\linearmodels\panel\model.py:1183: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


AbsorbingEffectError: 
The model cannot be estimated. The included effects have fully absorbed
one or more of the variables. This occurs when one or more of the dependent
variable is perfectly explained using the effects included in the model.

The following variables or variable combinations have been fully absorbed
or have become perfectly collinear after effects are removed:

          Earnings Per Share – Coefficient of Variation, Market Capitalization, Revenue - Actual, Enterprise Value, Number of Analysts, Recommendation - Mean (1-5), Recommendation - Mean (1-5).1, Net Income after Tax, Price Target - Mean, Price Target - Standard Deviation, Price Close, Enterprise Value To Sales (Daily Time Series Ratio), 3 Month Total Return, Volume, Earnings Per Share - Actual Surprise AbsVals

Set drop_absorbed=True to automatically drop absorbed variables.


In [ ]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Earnings Per Share – Coefficient of Variation":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary
